# 모델 구현 부분까지만 제출합니다.

# Resnet34

In [1]:
import torch
import torch.nn as nn
import torchsummary
import torch.onnx

c:\Users\K\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import torch.nn as nn


class Block(nn.Module):

    def __init__(self, architecture, is_shortcut):
        super(Block, self).__init__()
        self.num_repeat = architecture[0] # 첫번째 인자가 반복 횟수
        self.architecture = architecture[1:] # 나머지 인자가 CNN block 파라미터
        self.is_shorcut = is_shortcut
        self.shortcut = nn.Sequential()
        self.block = self._block(self.num_repeat, self.architecture)
        self.relu = nn.ReLU()

    
    def _block(self, num_repeat, architecture):
        layers = []

        for i in range(num_repeat):
            for j, (in_channels, out_channels, kernel_size, stride, padding) in enumerate(architecture):
                layers.append(nn.Conv2d(
                    in_channels=in_channels,
                    out_channels=out_channels,
                    kernel_size=kernel_size,
                    stride=stride,
                    padding=padding,
                    bias=False,
                ))
                layers.append(nn.BatchNorm2d(out_channels))
                # 가장 마지막 ReLU는 identity를 더한 후 적용한다.
                if not (((i + 1) == num_repeat) and ((j + 1) == len(architecture))):
                    layers.append(nn.ReLU())
                    
        return nn.Sequential(*layers)


    def forward(self, x):
        out = self.block(x)
        if self.is_shorcut:
            identity = self.shortcut(x)
            out += identity
        out =  self.relu(out)
        return out


class Resnet34(nn.Module):

    def __init__(self):
        super(Resnet34, self).__init__()
        self.architecture = [
            ["I", (3, 64, 7, 2, 3)], # [입력층을 의미하는 코드, 적용할 CNN인자]
            ["M", (3, 2)], # [최대풀링을 의미하는 코드, 적용할 MaxPool인자]
            [3, (64, 64, 3, 1, 1), (64, 64, 3, 1, 1)], # [블록 반복횟수, 적용할 CNN인자]
            [1, (64, 128, 3, 2, 1)], # [블록 반복횟수(이미지 사이즈 감소), 적용할 CNN인자]
            [4, (128, 128, 3, 1, 1), (128, 128, 3, 1, 1)],
            [1, (128, 256, 3, 2, 1)],
            [6, (256, 256, 3, 1, 1), (256, 256, 3, 1, 1)],
            [1, (256, 512, 3, 2, 1)],
            [3, (512, 512, 3, 1, 1), (512, 512, 3, 1, 1)],
        ]
        self.resnet34 = nn.Sequential(
            self._block(self.architecture),
            nn.AvgPool2d(1, 1),
            nn.Flatten(1, -1), # 배치까지 flatten을 적용하면 안되므로 (Batch(0), Channel(1), H(2), W(3)) 1부터 3까지만 flatten 함
            nn.Linear(512*7*7, 1),
            nn.Sigmoid(),
        )

    
    def _block(self, architecture):
        layers = []

        for x in architecture:
            # x의 첫번째 인자는 block의 종류를 결정
            if x[0] == "I": # 블록을 쌓는 코드, Input 레이어를 의미
                in_channels, out_channels, kernel_size, stride, padding = x[1]
                layers.append(nn.Conv2d(
                    in_channels=in_channels,
                    out_channels=out_channels,
                    kernel_size=kernel_size,
                    stride=stride,
                    padding=padding,
                    bias=False,
                ))

            elif x[0] == "M": # MaxPooling 레이어를 의미
                kernel_size, stride = x[1]
                layers.append(nn.MaxPool2d(
                    kernel_size=kernel_size,
                    stride=stride,
                ))

            elif x[0] > 1: # 블록을 쌓는 코드, 블록 반복 횟수를 의미
                block_architecture = x
                layers.append(Block(block_architecture, is_shortcut=True))

            elif x[0]== 1: # 이미지 사이즈를 줄이는 블록을 쌓는 코드, 블록 반복 횟수를 의미, 이때는 shortcut을 적용하지 않음
                block_architecture = x
                layers.append(Block(block_architecture, is_shortcut=False))
                
        return nn.Sequential(*layers)


    def forward(self, x):
        out = self.resnet34(x)
        return out

In [3]:
device = 'cuda'
model = Resnet34()
model = model.to(device)

In [4]:
torchsummary.summary(model, (3, 224, 224), device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
         MaxPool2d-2           [-1, 64, 55, 55]               0
            Conv2d-3           [-1, 64, 55, 55]          36,864
       BatchNorm2d-4           [-1, 64, 55, 55]             128
              ReLU-5           [-1, 64, 55, 55]               0
            Conv2d-6           [-1, 64, 55, 55]          36,864
       BatchNorm2d-7           [-1, 64, 55, 55]             128
              ReLU-8           [-1, 64, 55, 55]               0
            Conv2d-9           [-1, 64, 55, 55]          36,864
      BatchNorm2d-10           [-1, 64, 55, 55]             128
             ReLU-11           [-1, 64, 55, 55]               0
           Conv2d-12           [-1, 64, 55, 55]          36,864
      BatchNorm2d-13           [-1, 64, 55, 55]             128
             ReLU-14           [-1, 64,

In [21]:
input_names = ['Input']
output_names = ['Output']

x = torch.zeros(2, 3, 224, 224).to(device)
torch.onnx.export(model, x, 'Resnet34.onnx', input_names=input_names, output_names=output_names)

<img src='https://drive.google.com/uc?export=download&id=1jdvzWZLVZ7mwboIssAIpM-RfVg_L2GEM' width="" height ="" /><br>


# Resnet50

In [22]:
import torch
import torch.nn as nn


class Block(nn.Module):

    def __init__(self, architecture, is_shortcut):
        super(Block, self).__init__()
        self.num_repeat = architecture[0] # 첫번째 인자가 반복 횟수
        self.architecture = architecture[1:] # 나머지 인자가 CNN block 파라미터
        self.is_shorcut = is_shortcut
        if self.is_shorcut:
            # 1x1 Convolution filter
            # 마지막 CNN 파라미터의 입출력 필터 사이즈가 shortcut의 입출력 필터 사이즈와 일치함
            self.shortcut = nn.Conv2d(
                in_channels=architecture[-1][0],
                out_channels=architecture[-1][1],
                kernel_size=1,
                stride=1,
                padding=0,
                bias=False,
            )
        self.block = self._block(self.num_repeat, self.architecture)
        self.relu = nn.ReLU()

    
    def _block(self, num_repeat, architecture):
        layers = []

        in_channels = architecture[0][0]
        for i in range(num_repeat):
            for j, (_, out_channels, kernel_size, stride, padding) in enumerate(architecture):
                layers.append(nn.Conv2d(
                    in_channels=in_channels,
                    out_channels=out_channels,
                    kernel_size=kernel_size,
                    stride=stride,
                    padding=padding,
                    bias=False,
                ))
                layers.append(nn.BatchNorm2d(out_channels))
                # 가장 마지막 ReLU는 identity를 더한 후 적용한다.
                if not (((i + 1) == num_repeat) and ((j + 1) == len(architecture))):
                    layers.append(nn.ReLU())
                # 반복 사이클의 마지막에서 입출력 채널을 맞춰주기 위해 적용
                in_channels = out_channels

        return nn.Sequential(*layers)


    def forward(self, x):
        out = self.block(x)
        if self.is_shorcut:
            identity = self.shortcut(x)
            out += identity
        out =  self.relu(out)
        return out


class Resnet50(nn.Module):

    def __init__(self):
        super(Resnet50, self).__init__()
        self.architecture = [
            ["I", (3, 64, 7, 2, 3)], # [입력층을 의미하는 코드, 적용할 CNN인자]
            ["M", (3, 2)], # [최대풀링을 의미하는 코드, 적용할 MaxPool인자]
            [3, (64, 64, 1, 1, 0), (64, 64, 3, 1, 1), (64, 256, 1, 1, 0)], # [블록 반복횟수, 적용할 CNN인자]
            [1, (256, 128, 3, 2, 1)], # [블록 반복횟수(이미지 사이즈 감소), 적용할 CNN인자]
            [4, (128, 128, 1, 1, 0), (128, 128, 3, 1, 1), (128, 512, 1, 1, 0)],
            [1, (512, 256, 3, 2, 1)],
            [6, (256, 256, 1, 1, 0), (256, 256, 3, 1, 1), (256, 1024, 1, 1, 0)],
            [1, (1024, 512, 3, 2, 1)],
            [3, (512, 512, 1, 1, 0), (512, 512, 3, 1, 1), (512, 2048, 1, 1, 0)],
        ]
        self.resnet50 = nn.Sequential(
            self._block(self.architecture),
            nn.AvgPool2d(1, 1),
            nn.Flatten(1, -1), # 배치까지 flatten을 적용하면 안되므로 (Batch(0), Channel(1), H(2), W(3)) 1부터 3까지만 flatten 함
            nn.Linear(2048*7*7, 1),
            nn.Sigmoid(),
        )

    
    def _block(self, architecture):
        layers = []

        for x in architecture:
            # x의 첫번째 인자는 block의 종류를 결정
            if x[0] == "I": # 블록을 쌓는 코드, Input 레이어를 의미
                in_channels, out_channels, kernel_size, stride, padding = x[1]
                layers.append(nn.Conv2d(
                    in_channels=in_channels,
                    out_channels=out_channels,
                    kernel_size=kernel_size,
                    stride=stride,
                    padding=padding,
                    bias=False,
                ))

            elif x[0] == "M": # MaxPooling 레이어를 의미
                kernel_size, stride = x[1]
                layers.append(nn.MaxPool2d(
                    kernel_size=kernel_size,
                    stride=stride,
                ))

            elif x[0] > 1: # 블록을 쌓는 코드, 블록 반복 횟수를 의미
                block_architecture = x
                layers.append(Block(block_architecture, is_shortcut=True))

            elif x[0]== 1: # 이미지 사이즈를 줄이는 블록을 쌓는 코드, 블록 반복 횟수를 의미, 이때는 shortcut을 적용하지 않음
                block_architecture = x
                layers.append(Block(block_architecture, is_shortcut=False))

        return nn.Sequential(*layers)


    def forward(self, x):
        out = self.resnet50(x)
        return out

In [23]:
device = 'cuda'
model = Resnet50()
model = model.to(device)

In [24]:
torchsummary.summary(model, (3, 224, 224), device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
         MaxPool2d-2           [-1, 64, 55, 55]               0
            Conv2d-3           [-1, 64, 55, 55]           4,096
       BatchNorm2d-4           [-1, 64, 55, 55]             128
              ReLU-5           [-1, 64, 55, 55]               0
            Conv2d-6           [-1, 64, 55, 55]          36,864
       BatchNorm2d-7           [-1, 64, 55, 55]             128
              ReLU-8           [-1, 64, 55, 55]               0
            Conv2d-9          [-1, 256, 55, 55]          16,384
      BatchNorm2d-10          [-1, 256, 55, 55]             512
             ReLU-11          [-1, 256, 55, 55]               0
           Conv2d-12           [-1, 64, 55, 55]          16,384
      BatchNorm2d-13           [-1, 64, 55, 55]             128
             ReLU-14           [-1, 64,

In [25]:
input_names = ['Input']
output_names = ['Output']

x = torch.zeros(2, 3, 224, 224).to(device)
torch.onnx.export(model, x, 'Resnet50.onnx', input_names=input_names, output_names=output_names)

<img src='https://drive.google.com/uc?export=download&id=1jcBl2ujLBCpK_8KXMMksQYGiNGs5KJNb' width="" height ="" /><br>